In [57]:
import pandas as pd
import numpy as np
import re
from ast import literal_eval

전체 데이터 중 작품명/장르/방송채널/방송기간/방송시간/방송횟수/출연자 사용

In [58]:
file_path = r"C:\Users\sjung\Desktop\Data_Science_Lab\EDA_project\10th-EDA\Team_C\정제데이터\\"
meta_d= pd.read_csv(file_path+'메타_전처리1.csv',
                    converters={'장르': literal_eval})
meta_d.head()

,작품명,장르,방송사,전체회차,출연자,시작일,종료일,요일,월,화,...,한예리,한지민,한지현,한지혜,한채영,한효주,함은정,현빈,황민현,황정음
0,에이틴,"[웹드라마, 학원물, 로맨스]","V LIVE, 네이버TV, 유튜브, 페이스북",24,"['신예은', ' 이나은', ' 신승호', ' 김동희 ']",2018-07-01 0:00,2018-09-16 0:00,수일,0,0,...,0,0,0,0,0,0,0,0,0,0
1,최고의 치킨,[드라마],"iHQ DRAMA, MBN",12,"['박선호', ' 김소혜', ' 주우재 ']",2019-01-02 0:00,2019-02-07 0:00,수목,0,0,...,0,0,0,0,0,0,0,0,0,0
2,왼손잡이 아내,"[복수, 멜로]",KBS 2TV,3,"['이수경', ' 김진우', ' 진태현', ' 하연주', ' 이승연 ']",2019-01-02 0:00,2019-05-31 0:00,월화수목금,1,1,...,0,0,0,0,0,0,0,0,0,0
3,동네변호사 조들호 2: 죄와 벌,[법률 드라마],KBS 2TV,40,"['박신양', ' 고현정', ' 이민지', ' 변희봉 ']",2019-01-07 0:00,2019-03-26 0:00,월화,1,1,...,0,0,0,0,0,0,0,0,0,0
4,왕이 된 남자,[사극],tvN,16,"['여진구', ' 김상경', ' 이세영', ' 정혜영', ' 장광', ' 권해효 ']",2019-01-07 0:00,2019-03-04 0:00,월화,1,1,...,0,0,0,0,0,0,0,0,0,0


In [59]:
# index 316의 값이 이상하게 나오는 것 확인 (엑셀 파일상으로 318행)
meta_d.iloc[316,:]

작품명     12
장르      12
방송사     12
전체회차    12
출연자     12
        ..
한효주     12
함은정     12
현빈      12
황민현     12
황정음     12
Name: 316, Length: 1327, dtype: object

In [60]:
# index 316제거
meta_d = meta_d.drop(index = [316]).reset_index(drop = True)


In [65]:
genre_d = meta_d[['작품명','장르']]
genre_d.head(5)

,작품명,장르
0,에이틴,"[웹드라마, 학원물, 로맨스]"
1,최고의 치킨,[드라마]
2,왼손잡이 아내,"[복수, 멜로]"
3,동네변호사 조들호 2: 죄와 벌,[법률 드라마]
4,왕이 된 남자,[사극]


## strip을 통해 장르별 띄어쓰기 삭제하기

In [66]:
for i in range(len(genre_d)):
    row_data = genre_d.loc[i, '장르']
    for idx, data_in_list in enumerate(row_data):
        genre_d.loc[i, '장르'][idx] = data_in_list.strip()


# 장르 원핫 인코딩

In [75]:
dummies_a = pd.get_dummies(genre_d['장르'].apply(pd.Series).stack()).sum(level=0)

genre_d = genre_d.join(dummies_a)
genre_d.columns

C:\Users\sjung\AppData\Local\Temp\ipykernel_16884\3116314659.py:1: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  dummies_a = pd.get_dummies(genre_d['장르'].apply(pd.Series).stack()).sum(level=0)


Index(['작품명', '장르', 'SF', 'SF 시리즈', '[1] 로맨스', '가족', '감성', '격정멜로', '경제', '경찰',
       ...
       '현대 판타지', '형사', '형사물', '호러', '회귀', '회사 드라마', '휴머니즘', '휴먼', '휴먼 판타지 드라마',
       '힐링'],
      dtype='object', length=126)

# 데이터 출력

In [76]:
genre_d

,작품명,장르,SF,SF 시리즈,[1] 로맨스,가족,감성,격정멜로,경제,경찰,...,현대 판타지,형사,형사물,호러,회귀,회사 드라마,휴머니즘,휴먼,휴먼 판타지 드라마,힐링
0,에이틴,"[웹드라마, 학원물, 로맨스]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,최고의 치킨,[드라마],0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,왼손잡이 아내,"[복수, 멜로]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,동네변호사 조들호 2: 죄와 벌,[법률 드라마],0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,왕이 된 남자,[사극],0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
388,넘버스 : 빌딩숲의 감시자들,"[오피스, 복수, 범죄, 스릴러]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
389,악귀,"[판타지, 미스터리, 스릴러]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
390,아씨두리안,"[SF, 드라마, 로맨스]",1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
391,가슴이 뛴다,"[공포, 로맨스, 코미디]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [79]:
saving_path = r'C:\Users\sjung\Desktop\Data_Science_Lab\EDA_project\10th-EDA\Team_C\정제데이터\\'
genre_d.to_csv(saving_path + '장르만_다시_처리.csv', index=False, encoding='utf-8-sig')